Goal: with superusertoken; document ID and part IDS find all lines, where there are versions, extract all those versions and note the respective line ID down <br7>
PREREQUISITES: Levensthein module

In [258]:
import requests
import json
import Levenshtein
import tablib 

document/37/part/3965/

Needs: TOKEN; 

In [259]:
token = '81f07dae0233034e62ef3e4881bed4f936e8258a'

versionlist =[]
overview = []

response = requests.get('http://143.50.30.29:8080/api/', headers={'Authorization': 'Token ' + token})
print(response)

<Response [200]>


In [260]:

def type_check(token, documentid, partsid):
    """This function tests, if the parameters have the right type. It will be called in the version extractor function to check for the right types and 
    and raises errors, when needed"""
    if type(token) != str:
        raise TypeError ('Tokenparameter must be a string!')
    elif type(documentid) != int:
        raise TypeError ('DocumentID must be an integer!')
    elif type(partsid) != int:
        raise TypeError ('Tokenparameter must be an integer!')
    else: 
        print ('Parameters are all good!')

In [261]:

def line_num(token, documentid, partsid, pagenum):
    """Gets the id of all the lines on a page and stores them in an empty list. If the statuscode 404 appears, it stopps and prints an ending message. Afterwards 
    the API points of the sites will be opened in a loop and if there are versions, those will be saved"""
    linenum=[]    
    api_url = 'http://143.50.30.29:8080/api/documents/'
    url = api_url + str(documentid) + "/parts/" + str(partsid) + '/lines/?page=' + str(pagenum)
    doc_response = requests.get(url, headers={'Authorization': 'Token ' + token})   
    json = doc_response.json()
    print('Grabbing page....')
    if doc_response.status_code != 404:
        for i in json['results']:
            linenum.append(i['pk'])
            #print(i['pk'])
        pagenum = pagenum+1    
        #print(pagenum)
        line_num(token, documentid, partsid, pagenum)      
    else:
        print('All pagenumbers collected. Last page reached.')
    for i in linenum:
        url = api_url + str(documentid) + "/parts/" + str(partsid) + '/lines/' +  str(i)    
        resp = requests.get(url, headers={'Authorization': 'Token ' + token})
        data = resp.json()
        linevers = []  
        order = data['order'] + 1
        linevers.append(order)        
        for transcription in data['transcriptions']:          
           if transcription['versions'] != []:
               current = transcription['content'] #version data to list 
               current_version = '\n'+ "Current Version:    " + transcription['content']   + '\n'             
               filename = 'document-' + str(documentid) + '-part-' + str(partsid) + '.txt'
               with open(filename,  'a', encoding="utf-8") as f: 
                    f.write(current_version)  
               linevers.append(current)
               for versions in transcription['versions']:                 
                   a = versions['data']['content']
                   linevers.append(a)                   
                   version = 'Older Version:      ' + versions['data']['content'] + '\n'
                   date = '------> Date:        '+ versions['updated_at'] + ' ' + '\n'
                   #print('Versionscontent: ', versions['data']['content'], ' Author:     ', versions['author'], '  Date:    ', versions['updated_at'])         
                   with open(filename,  'a', encoding="utf-8") as f: 
                        f.write(version + date)   
               versionlist.append(linevers)

In [262]:

"""extracts versions as a dirty txt file and fills in lists for further processing"""
def version_extractor(token: str, documentid: int, partsid: int):      
    type_check(token, documentid, partsid)
    api_url = 'http://143.50.30.29:8080/api/documents/'
    name = api_url + str(documentid)
    overview.append(name)
    line_num(token, documentid, partsid, 1)
    print('Data extracted and saved as TXT.')
    print('Starting CSV building...')

In [263]:
version_extractor(token, 10, 1502) # get all line numbers 

Parameters are all good!
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
Grabbing page....
All pagenumbers collected. Last page reached.
Data extracted and saved as TXT.
Starting CSV building...


/lines --> hat count aller aller lines; 10 pro seite; results/pk is die jeweilige line num; 
in den lines gibts versions 

6

In [222]:
print(overview)

['http://143.50.30.29:8080/api/documents/4']


In [210]:
"""for i in linenum:
    url = api_url + str(37) + "/parts/" + str(3965) + '/lines/' +  str(i)
    
    resp = requests.get(url, headers={'Authorization': 'Token ' + token})
    data = resp.json()
    for transcription in data['transcriptions']:
       if transcription['versions'] != []:
           #print(transcription)
           print('_________________________________________')
           print ('Current Version:    ', transcription['content'], 'Author:      ', transcription['version_author'] )
           for versions in transcription['versions']:
               print('Versionscontent: ', versions['data']['content'], ' Author:     ', versions['author'], '  Date:    ', versions['updated_at'])   
           
           
           
          # print('empty')

           
             
         
   # print(check['transcriptions'])
    #print(api_url + str(37) + "/parts/" + str(3965) + '/lines/' +  str(i))"""

'for i in linenum:\n    url = api_url + str(37) + "/parts/" + str(3965) + \'/lines/\' +  str(i)\n    \n    resp = requests.get(url, headers={\'Authorization\': \'Token \' + token})\n    data = resp.json()\n    for transcription in data[\'transcriptions\']:\n       if transcription[\'versions\'] != []:\n           #print(transcription)\n           print(\'_________________________________________\')\n           print (\'Current Version:    \', transcription[\'content\'], \'Author:      \', transcription[\'version_author\'] )\n           for versions in transcription[\'versions\']:\n               print(\'Versionscontent: \', versions[\'data\'][\'content\'], \' Author:     \', versions[\'author\'], \'  Date:    \', versions[\'updated_at\'])   \n           \n           \n           \n          # print(\'empty\')\n\n           \n             \n         \n   # print(check[\'transcriptions\'])\n    #print(api_url + str(37) + "/parts/" + str(3965) + \'/lines/\' +  str(i))'

In [211]:
"""liste = []

for z in range(len(versionlist)):
    liste.append(z)

print(liste)"""

'liste = []\n\nfor z in range(len(versionlist)):\n    liste.append(z)\n\nprint(liste)'

In [264]:
import Levenshtein
print(len(versionlist))
DBLIST = tablib.Dataset(headers=['Current Version','Initial Version', 'Line number', 'Levenshtein Distance', 'Similarity Ratio'])
for index in range(len(versionlist)):
    order = versionlist[index][0]
    currentversion = versionlist[index][1]
    startversion = versionlist[index][-1]
    dist_calc = Levenshtein.distance(currentversion, startversion)    
    similarity_ratio = Levenshtein.ratio(currentversion, startversion)    
    DBLIST.append([currentversion, startversion, order, dist_calc, similarity_ratio])

90


In [265]:
with open('glossit.csv',  'w', encoding="utf-8") as f:
    f.write(DBLIST.export('csv'))

* Document Name
* Page ID
* Link 
* wie viele lines gesamt und wie viele mussten geändert werden
* Prozentsatz 
* erste und letzte iteration
* levensthein Distance zwischen den iterationen


75.63451776649747


In [120]:
edit_dist

2

* feinschliff - es wird nur die document ID gebraucht um einen umfassenden overview zu geben
* 